In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

HOST = "localhost"
PORT = 4223
SS1_UID = "6wVow5"
# SS2_UID = "6CQ4nA"
LC1_UID = 'KiG'
# LC2_UID = 'Kgx'
import time

from tinkerforge.ip_connection import IPConnection
from tinkerforge.brick_silent_stepper import BrickSilentStepper
from tinkerforge.bricklet_load_cell_v2 import BrickletLoadCellV2
import numpy as np
from tqdm import tqdm
import pandas as pd
import time

SAVE_FILENAME = "PLACEHOLDER.csv"


ipcon = IPConnection() # Create IP connection
ss1 = BrickSilentStepper(SS1_UID, ipcon) # Create device object
# ss2 = BrickSilentStepper(SS2_UID, ipcon) # Create device object
lc1 = BrickletLoadCellV2(LC1_UID, ipcon) # Create device object
# lc2 = BrickletLoadCellV2(LC2_UID, ipcon) # Create device object
ipcon.connect(HOST, PORT) # Connect to brickd

In [ ]:
PITCH = 5  # millimeters / turn
GEAR_RATIO = 1  # Assuming no gear ratio? https://www.omc-stepperonline.com/download/11HS12-0956D.pdf
MICRO_STEPS_PER_REV, RESOLUTION_SETTING = (
    256,
    ss1.STEP_RESOLUTION_256,
)  # 256 micro-steps per macro-step
MACRO_STEPS_PER_REV = 200  # 200 macro-steps per revolution (1.8 degree step angle)
STEPS_PER_REV = GEAR_RATIO * MACRO_STEPS_PER_REV * MICRO_STEPS_PER_REV
FINAL_DISTANCE = 5  # mm
MEASURE_FREQ = 0.1  # mm per measurement


MAX_VELOCITY = 0.1  # mm / s
MAX_STEPS_PER_SECOND = (
    MAX_VELOCITY * STEPS_PER_REV
) / PITCH  # Velocity: steps / sec
FINAL_STEPS = (FINAL_DISTANCE * STEPS_PER_REV) / PITCH
MEASURE_FREQ_IN_STEPS = (STEPS_PER_REV * MEASURE_FREQ) / PITCH



In [ ]:
df = pd.DataFrame(columns = ['Time', 'Force (Newtons)', 'Distance (millimeters)'])

ss1.enable() 
ss1.set_current_position(0) # Initialize to 0
ss1.set_motor_current(800) # 800 mA
ss1.set_step_configuration(RESOLUTION_SETTING, True) # 1 / RESOLUTION_SETTING steps (interpolated)
ss1.set_max_velocity(MAX_STEPS_PER_SECOND) # Velocity: steps/s
ss1.set_speed_ramping(500, 5000) # Fast deacceleration (5000 steps/s^2) for stopping

# Initialize Tar & Set Stepper position to 0
lc1_tar = np.mean([lc1.get_weight() for i in range(1000)])
# lc2_tar = np.mean([lc2.get_weight() for i in range(1000)])

print(lc1_tar)

In [ ]:
# LETS RUN IT! NOTE: slapping on the negative sign sets the stepper to stretch
ss1.set_steps(-FINAL_STEPS)


In [ ]:
from IPython.display import clear_output

STARTING_TIME = time.time_ns() // 1_000_000
total_steps, total_distance, total_time, total_newtons, idx = 0, 0, 0, 0, 0

while total_distance < FINAL_DISTANCE:
    total_steps = -ss1.get_current_position()
    total_time = (time.time_ns() // 1_000_000) - STARTING_TIME
    total_distance = total_steps / STEPS_PER_REV * PITCH
    lc1_weight = lc1_tar - lc1.get_weight()  # gramspp
    # lc2_weight = (lc2.get_weight()) - lc2_tar
    # NOTE: Yitong calibrated the load cells to 10,000x
    # sensitivity.... (with a 50gram weight said it was 500000 grams)
    # total_weight = (lc1_weight + lc2_weight) / 10000
    total_weight = (lc1_weight ) / 10000

    total_newtons = -total_weight / 1000 * 9.80665  # m/s^2
    clear_output(wait=True)  # Clear the previous output

    display(
        f"time {total_time:.0f}, lc1_weight {lc1_weight:.1f}, Total Time {total_time:.1f}, Total Weight {total_weight:.5f}, Total Newtons {total_newtons:.5f}"
    )

    # display(f'{idx} Total Distance {total_distance:.5f}, Total Force {total_newtons:.5f}, Total Time {total_time:.5f}')

    # Create a new row as a DataFrame
    new_row = pd.DataFrame(
        {
            "Time": [total_time],
            "Distance (millimeters)": [total_distance],
            "Force (Newtons)": [total_newtons],
        }
    )

    # Use pd.concat to add the new row to the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

    # time.sleep(0.01)
    # clear_output(wait=True)
    df.to_csv(SAVE_FILENAME)

In [ ]:
time.sleep(2) # Wait for motor to actually stop
ss1.disable()
ipcon.disconnect()